In [1]:
#!pip install --quiet glvis
import numpy as np
import matplotlib.pyplot as plt
from  bfp import *
import mfem.ser as mfem
#from mfem.common.arg_parser import ArgParser
from glvis import glvis, GlvisData

import os
import h5py

#import math
#import mpi4py.MPI as MPI
#print(MPI.Get_version())

In [2]:
def create_2D_mesh(nx, ny, x_start, x_end, y_start, y_end):
    """Creates a 2D mesh with the specified intervals and coordinate ranges.

    Args:
        nx (int): Number of intervals in the x-direction.
        ny (int): Number of intervals in the y-direction.
        x_start (float): Starting x-coordinate.
        x_end (float): Ending x-coordinate.
        y_start (float): Starting y-coordinate.
        y_end (float): Ending y-coordinate.

    Returns:
        mesh: The updated mesh with vertex coordinates set accordingly.
    """

    # Generate equally spaced coordinates for x and y
    x_coords = np.linspace(x_start, x_end, nx + 1)
    y_coords = np.linspace(y_start, y_end, ny + 1)
    
    # Create the mesh with initial x and y values set to 0
    mesh = mfem.Mesh(nx, ny, "QUADRILATERAL", True, 0.0, 0.0)
    
    # Retrieve the vertex array from the mesh
    verts = mesh.GetVertexArray()
    
    # Check if the number of vertices is as expected: (nx+1) * (ny+1)
    expected_num = (nx + 1) * (ny + 1)
    num_verts = mesh.GetNV()

    if num_verts != expected_num:
        print("Warning: Unexpected number of vertices! ({} != {})".format(num_verts, expected_num))
    
    # Update the vertex coordinates; vertices are stored in row-major order
    k = 0
    for j in range(ny + 1):
        for i in range(nx + 1):
            verts[k][0] = x_coords[i]
            verts[k][1] = y_coords[j]
            k += 1
    """
    # Define the target directory and file name
    target_dir = os.path.join(os.getcwd(), 'mesh', 'usr')
    file_name = f'{nx}x{ny}_2D.mesh'
    file_path = os.path.join(target_dir, file_name)
    
    # Check if the target directory exists; if not, create it
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)
        print(f"Directory '{target_dir}' was created.")

    # Check if the file already exists
    if not os.path.exists(file_path):
        # If the file does not exist, write the mesh to the file
        mesh.Print(file_path)
        print(f"File '{file_path}' was successfully created.")
    else:
        print(f"File '{file_path}' already exists.")
    """
    return mesh

In [3]:
class TotalXSCoefficient(mfem.PyCoefficient):
    """Coefficient for the total cross-section Σ_t(E).

    This coefficient maps the normalized energy coordinate (x[1]) in [0, 1] to the
    corresponding total cross-section value. Specifically, a value of y = 0 corresponds to
    E = E_start and y = 1 corresponds to E = E_end. If a constant is provided 
    (e.g., 1), the coefficient returns this constant at all points.

    Examples:
        TotalXSCoefficient(1)
            # Always returns 1.

        TotalXSCoefficient(xs_t_data, E_start=0.0, E_end=1.0)
            # Returns the interpolated value from xs_t_data.
    """

    def __init__(self, xs_t_data, E_start=None, E_end=None):
        super(TotalXSCoefficient, self).__init__()
        if isinstance(xs_t_data, (int, float)):
            self.constant = True
            self.constant_value = float(xs_t_data)
        else:
            self.constant = False
            self.xs_t_data = xs_t_data
            self.E_start = E_start
            self.E_end = E_end

    def EvalValue(self,ip):
        """Evaluates the total cross-section at a given energy coordinate.

        This method converts the normalized energy coordinate found in x[1] to the actual 
        energy value and determines the corresponding energy group to return the associated 
        total cross-section value. 

        Args:
        ip (list or array-like): A coordinate array where ip[1] is the normalized energy 
                (in the range [0, 1]).

        Returns:
            float: The total cross-section value corresponding to the computed energy.
        """
        if self.constant:
            return self.constant_value

        y = (ip[1] - self.E_start) / (self.E_end - self.E_start)
        E = self.E_start + y * (self.E_end - self.E_start)
        n_groups = len(self.xs_t_data)
        group = min(n_groups - 1,
                    int((E - self.E_start) / (self.E_end - self.E_start) * n_groups))
        return float(self.xs_t_data[group])


class ScatteringXSCoefficient(mfem.PyCoefficient):
    """Coefficient for the scattering cross-section Σ_s(E).

    This coefficient maps the normalized energy coordinate (x[1]) in [0, 1] to the
    corresponding cross-section value. Specifically, a value of y = 0 corresponds to
    E = E_start and y = 1 corresponds to E = E_end. If a constant is provided 
    (e.g., 1), the coefficient returns this constant at all points.

    Examples:
        ScatteringXSCoefficient(1)
            # Always returns 1.

        ScatteringXSCoefficient(xs_s_data, E_start=0.0, E_end=1.0)
            # Returns the interpolated value from xs_s_data.
    """

    def __init__(self, xs_s_data, E_start=None, E_end=None):
        super(ScatteringXSCoefficient, self).__init__()
        if isinstance(xs_s_data, (int, float)):
            self.constant = True
            self.constant_value = float(xs_s_data)
        else:
            self.constant = False
            self.xs_s_data = xs_s_data
            self.E_start = E_start
            self.E_end = E_end

    def EvalValue(self, ip):
        """Evaluates the scattering cross-section at a given normalized energy coordinate.

        This method converts the normalized energy coordinate found in x[1] to the actual 
        energy value and determines the corresponding energy group to return the associated 
        scattering cross-section value. 

        Args:
            ip (list or array-like): A coordinate array where ip[1] is the normalized energy 
                in the range [0, 1].

        Returns:
            float: The scattering cross-section value for the computed energy.
        """
        if self.constant:
            return self.constant_value

        y = (ip[1] - self.E_start) / (self.E_end - self.E_start)
        E = self.E_start + y * (self.E_end - self.E_start)
        n_groups = len(self.xs_s_data)
        group = min(n_groups - 1,
                    int((E - self.E_start) / (self.E_end - self.E_start) * n_groups))
        return float(self.xs_s_data[group])

class StoppingPowerCoefficient(mfem.PyCoefficient):
    """Coefficient for the stopping power S(E).

    This coefficient maps the normalized energy coordinate (ip[1]) in the interval [0, 1]
    to the corresponding stopping power S(E). Specifically, a value of y = 0 corresponds to 
    E = E_start, and a value of y = 1 corresponds to E = E_end. If a constant is provided 
    (e.g., 1), the coefficient returns this constant at all points.

    Examples:
        StoppingPowerCoefficient(1)
            # Always returns 1.

        StoppingPowerCoefficient(S_data, E_start=0.0, E_end=1.0)
            # Returns the interpolated value from S_data.
    """

    def __init__(self, S_data, E_start=None, E_end=None):
        super(StoppingPowerCoefficient, self).__init__()
        if isinstance(S_data, (int, float)):
            self.constant = True
            self.constant_value = float(S_data)
        else:
            self.constant = False
            self.S_data = S_data
            self.E_start = E_start
            self.E_end = E_end

    def EvalValue(self, ip):
        """Evaluates the stopping power at a given normalized energy coordinate.

        This method converts the normalized energy coordinate found in ip[1] to the actual 
        energy value and determines the corresponding energy group to return the associated 
        stopping power value. 

        Args:
            ip (list or array-like): A coordinate array where ip[1] is the normalized energy 
                (in the range [0, 1]).

        Returns:
            float: The stopping power value corresponding to the computed energy.
        """
        if self.constant:
            return self.constant_value
        y = (ip[1] - self.E_start) / (self.E_end - self.E_start)
        E = self.E_start + y * (self.E_end - self.E_start)
        n_groups = len(self.S_data)
        group = min(n_groups - 1,
                    int((E - self.E_start) / (self.E_end - self.E_start) * n_groups))
        return float(self.S_data[group])

class StoppingPowerDerivativeCoefficient(mfem.PyCoefficient):
    """Coefficient for the derivative of the stopping power S(E), i.e., S'(E).

    This coefficient maps the normalized energy coordinate (x[1]) in the interval [0, 1]
    to the corresponding derivative of the stopping power S(E). A value of y = 0 corresponds 
    to E = E_start and y = 1 corresponds to E = E_end. If a constant is provided 
    (e.g., 1), the coefficient returns this constant at all points.

    Examples:
        StoppingPowerDerivativeCoefficient(1)
            # Always returns 1.

        StoppingPowerDerivativeCoefficient(dS_data, E_start=0.0, E_end=1.0)
            # Returns the interpolated value from data_array.
    """

    def __init__(self, dS_data, E_start=None, E_end=None):
        super(StoppingPowerDerivativeCoefficient, self).__init__()
        if isinstance(dS_data, (int, float)):
            self.constant = True
            self.constant_value = float(dS_data)
        else:
            self.constant = False
            self.dS_data = dS_data
            self.E_start = E_start
            self.E_end = E_end

    def EvalValue(self, ip):
        """Evaluates the derivative of the stopping power at a given normalized energy coordinate.

        This method converts the normalized energy coordinate found in ip[1] to the actual 
        energy value and determines the corresponding energy group to return the associated 
        derivative of the stopping power value. 

        Args:
            ip (list or array-like): A coordinate array where ip[1] is the normalized energy 
                (in the range [0, 1]).

        Returns:
            float: The derivative of the stopping power corresponding to the computed energy.
        """
        if self.constant:
            return self.constant_value
        y = (ip[1] - self.E_start) / (self.E_end - self.E_start)
        E = self.E_start + y * (self.E_end - self.E_start)
        n_groups = len(self.dS_data)
        group = min(n_groups - 1,
                    int((E - self.E_start) / (self.E_end - self.E_start) * n_groups))
        return float(self.dS_data[group])


class InflowCoefficientSN(mfem.PyCoefficient):
    """Coefficient for an inflow boundary condition in (x,E) space with SN angular dependence.

    This class returns an inflow flux value that depends on the discrete ordinate
    (angular) parameter \(\mu\). On the left boundary:
      - If \(\mu > 0\), the inflow flux is applied (e.g., set to a prescribed value).
      - If \(\mu < 0\), the incoming flux is zero.
    
    This is useful in SN (discrete ordinates) methods where the transport equation is
    solved separately for each angular direction.
    
    Attributes:
        in_flux (float): The constant inflow flux value to be used when \(\mu > 0\).
        mu (float): The discrete ordinate (angular direction) for which this coefficient is used.
    """

    def __init__(self, in_flux, mu):
        """Initializes the SN inflow coefficient.
        
        Args:
            in_flux (float): The constant inflow flux value for \(\mu > 0\).
            mu (float): The discrete angular direction value.
        """
        super(InflowCoefficientSN, self).__init__()
        self.in_flux = in_flux
        self.mu = mu

    def EvalValue(self, ip):
        """Evaluates the inflow coefficient at a given integration point.
        
        In this implementation, the integration point is interpreted in (x,E) space.
        The angular parameter \(\mu\) is stored in the class. The function returns
        the prescribed inflow flux if \(\mu > 0\) and zero otherwise.
        
        Args:
            ip (mfem.IntegrationPoint or list[float]): The integration point coordinates,
                where ip[0] is the spatial coordinate \(x\) and ip[1] is the energy \(E\).
                The angular information is not included in ip.
        
        Returns:
            float: The inflow flux value if \(\mu > 0\); otherwise, 0.0.
        """
        # Since the integration point ip does not include the angular variable,
        # we use the stored self.mu to determine the flux.
        if self.mu > 0:
            return self.in_flux
        else:
            return 0.0


class QCoefficient(mfem.PyCoefficient):
    """Coefficient for the energy dependent source term Q(E). ((will be updated for Q(x,E)))

    This coefficient maps the normalized energy coordinate (ip[1]) in the interval [0, 1]
    to the corresponding the energy dependent source Q(E). Specifically, a value of y = 0 
    corresponds to E = E_start, and a value of y = 1 corresponds to E = E_end. If a constant 
    is provided 
    (e.g., 1), the coefficient returns this constant at all points.

    Examples:
        StoppingPowerCoefficient(1)
            # Always returns 1.

        StoppingPowerCoefficient(S_data, E_start=0.0, E_end=1.0)
            # Returns the interpolated value from S_data.
    """
    def __init__(self, Q_data, E_start=None, E_end=None):
        super(QCoefficient, self).__init__()
        if isinstance(Q_data, (int, float)):
            self.constant = True
            self.constant_value = float(Q_data)
        else:
            self.constant = False
            self.Q_data = Q_data
            self.E_start = E_start
            self.E_end = E_end

    def EvalValue(self, ip):
        """Evaluates the energy dependent source term Q(E) at a given normalized energy coordinate.

        For non-constant Q_data, this method converts the normalized energy coordinate ip[1] to the 
        corresponding energy value and returns the Q value for the associated energy group. 
        If Q_data is constant, it always 
        returns this constant value.

        Args:
            ip (list or array-like): A coordinate array where ip[1] is the normalized energy 
                (in the range [0, 1]).

        Returns:
            float: The energy dependent source value corresponding to the computed energy.
        """

        if self.constant:
            return self.constant_value

        y = (ip[1] - self.E_start) / (self.E_end - self.E_start)
        E = self.E_start + y * (self.E_end - self.E_start)
        n_groups = len(self.Q_data)
        group = min(n_groups - 1, int((E - self.E_start) / (self.E_end - self.E_start) * n_groups))
        return float(self.Q_data[group])


class EnergyDependentCoefficient(mfem.PyCoefficient):
    """Energy-dependent coefficient using either a constant or a one-dimensional array.

    This coefficient maps a normalized energy coordinate (ip[1] in [0, 1])
    to a value by linearly interpolating data over an interval defined by E_start and E_end.
    If a constant is provided (e.g., 1), it is converted to an array (with two points)
    and then processed identically to an array input.

    Examples:
        EnergyDependentCoefficient(1)
            # Always returns 1.

        EnergyDependentCoefficient(data_array, E_start=0.0, E_end=1.0)
            # Returns the interpolated value from data_array.
    """

    def __init__(self, data, E_start=None, E_end=None):
        super(EnergyDependentCoefficient, self).__init__()
        if isinstance(data, (int, float)):
            self.constant = True
            self.constant_value = float(data)
        else:
            self.constant = False
            self.data = data
            self.E_start = E_start
            self.E_end = E_end

    def EvalValue(self,ip):
        """Evaluates the coefficient at a given normalized energy coordinate.

        This method converts the normalized energy coordinate (ip[1]) to a physical energy
        value, determines the corresponding index (group) in the data array via linear 
        interpolation, and returns the associated coefficient value.

        Args:
            ip (list or array-like): A coordinate array where ip[1] is the normalized energy 
                (in the range [0, 1]).

        Returns:
            float: The interpolated coefficient value corresponding to the computed energy.
        """
        if self.constant:
            return self.constant_value

        y = (x[1] - self.E_start) / (self.E_end - self.E_start)
        E = self.E_start + y * (self.E_end - self.E_start)
        n_groups = len(self.data)
        group = min(n_groups - 1, int((E - self.E_start) / (self.E_end - self.E_start) * n_groups))
        return float(self.data[group])

class XDependentCoefficient(mfem.PyCoefficient):
    """X-dependent coefficient using either a constant or a one-dimensional array.

    This coefficient maps a normalized x coordinate (ip[0] in [0, 1])
    to a value by linearly interpolating data over an interval defined by x_start and x_end.
    If a constant is provided (e.g., 1), it is converted to an array (with two points)
    and then processed identically to an array input.

    Examples:
        XDependentCoefficient(1)
            # Always returns 1.

        XDependentCoefficient(data_array, x_start=0.0, x_end=1.0)
            # Returns the interpolated value from data_array.
    """

    def __init__(self, data, x_start=None, x_end=None):
        super(XDependentCoefficient, self).__init__()
        if isinstance(data, (int, float)):
            self.constant = True
            self.constant_value = float(data)
        else:
            self.constant = False
            self.data = data
            self.x_start = x_start
            self.x_end = x_end

    def EvalValue(self, ip):
        """Evaluates the coefficient at a given normalized x coordinate.

        This method converts the normalized x coordinate (ip[0]) to a physical x
        value, determines the corresponding index (group) in the data array via linear 
        interpolation, and returns the associated coefficient value.

        Args:
            ip (list or array-like): A coordinate array where ip[0] is the normalized x 
                (in the range [0, 1]).

        Returns:
            float: The interpolated coefficient value corresponding to the computed x.
        """
        if self.constant:
            return self.constant_value

        y = (ip[0] - self.x_start) / (self.x_end - self.x_start)
        x_val = self.x_start + y * (self.x_end - self.x_start)
        n_groups = len(self.data)
        group = min(n_groups - 1, int((x_val - self.x_start) / (self.x_end - self.x_start) * n_groups))
        return float(self.data[group])
    

class VelocityCoefficient2(mfem.VectorPyCoefficientBase):
    """Velocity vector coefficient for the transport equation.

    Defines a velocity vector:
        v(x) = [μ, S(E)]

    Attributes:
        mu (float): Scalar value, e.g., discrete ordinate.
        S_coef (Coefficient): Coefficient object used to evaluate S(E) at given points.
    """

    def __init__(self, mu, S_coeff):
        mfem.VectorPyCoefficientBase.__init__(self, 2, 0)
        self.mu = mfem.array(mu)
        self.S_coeff = S_coeff

    def EvalValue(self, x):
        """Evaluates the velocity vector at a given integration point.

        Args:
            V (array_like): Output vector of size 2 to store the velocity.
            ip (IntegrationPoint): Integration point where the evaluation occurs.
        """

        return [self.mu, self.S_coeff(x)]
    

def assign_boundary_attributes(mesh, x_start, E_start, x_end, tol=1e-6):
    """
    Assigns boundary attributes to the mesh:
      - If any vertex of a boundary element is near (x_start, E_start), assign attribute 1.
      - Else if any vertex of a boundary element is near x_end, assign attribute 2.
      - Otherwise, leave the attribute as 0.
    
    Parameters:
      mesh    : The mfem.Mesh object.
      x_start : The x-coordinate for the left (inflow) boundary.
      E_start : The energy coordinate for the left boundary.
      x_end   : The x-coordinate for the right (outflow) boundary.
      tol     : Tolerance for comparing coordinates.
    """
    vertex_array = mesh.GetVertexArray()
    
    # Initialize all boundary element attributes to 0.
    for i in range(mesh.GetNBE()):
        mesh.SetBdrAttribute(i, 0)
    
    # Loop over each boundary element and assign attributes.
    for i in range(mesh.GetNBE()):
        v_indices = mesh.GetBdrElementVertices(i)
        for idx in v_indices:
            x_coord = vertex_array[idx][0]
            E_coord = vertex_array[idx][1]
            if abs(x_coord - x_start) < tol and abs(E_coord - E_start) < tol:
                mesh.SetBdrAttribute(i, 1)
                break
            elif abs(x_coord - x_end) < tol:
                mesh.SetBdrAttribute(i, 2)
                break


def get_marker_for_mu(mesh, mu):
    """
    Returns a marker array based on the mesh boundary attributes and the sign of mu.
      - If mu > 0, only boundaries with attribute 1 (left/inflow) are marked.
      - If mu < 0, only boundaries with attribute 2 (right/inflow) are marked.
    
    Parameters:
      mesh : The mfem.Mesh object (its boundary attributes should already be set).
      mu   : The angular variable.
    
    Returns:
      marker (mfem.intArray): An integer array with 1's where the appropriate boundary
                              (inflow) is active and 0's elsewhere.
    """
    num_bdr = mesh.bdr_attributes.Size()
    marker = mfem.intArray(num_bdr)
    marker.Assign(0)
    for i in range(num_bdr):
        attr = mesh.bdr_attributes[i]
        if mu > 0 and attr == 1:
            marker[i] = 1
        elif mu < 0 and attr == 2:
            marker[i] = 1
    return marker


class VelocityCoefficient(mfem.VectorPyCoefficient):
    """
    A simple vector coefficient:
       v(x) = [mu, S(E)]
    that returns a Python list in EvalValue.
    """
    def __init__(self, mu, S_arr, E_start, E_end):
        super(VelocityCoefficient, self).__init__(2)  # 2D vector
        self.mu = mu
        self.S_arr = S_arr
        self.E_start = E_start
        self.E_end   = E_end

    def EvalValue(self, x):
        """
        x is the coordinate array, e.g. [x_coord, E_coord].
        Returns [mu, S(E)].
        """
        comp0 = self.mu
        E = self.E_start + x[1]*(self.E_end - self.E_start)
        
        n_groups = len(self.S_arr)
        idx = min(n_groups - 1,
                  int((E - self.E_start)/(self.E_end - self.E_start) * n_groups))
        
        comp1 = self.S_arr[idx]
        
        return [comp0, comp1]


class constant(mfem.PyCoefficient):
    def __init__(self, mu):
        super(constant, self).__init__()
        self.mu = mu
    def EvalValue(self, x):
        return 0.01

In [6]:
# Set parameters
nx = 10
nE = 30
x_start = 0.0
x_end = 0.3
E_start = 1
E_end = 0.01
N_ang = 4
order = 1


mesh = create_2D_mesh(nx, nE, x_start, x_end, E_start, E_end)
dim = mesh.Dimension()
#set_boundary_attribute(mesh, -1, x_start, E_start, x_end, tol=1e-8)
"""
vertex_array = mesh.GetVertexArray()
for i in range(mesh.GetNBE()):
    v_indices = mesh.GetBdrElementVertices(i)
    idx0 = v_indices[0]
    vx = vertex_array[idx0][0]
    vE = vertex_array[idx0][1]
    attr = mesh.GetBdrAttribute(i)
    print(f"Boundary element {i}: Vertex = ({vx}, {vE}), Attribute = {attr}")
"""

fec = mfem.DG_FECollection(order, dim)
fes = mfem.FiniteElementSpace(mesh, fec)
Size = fes.GetTrueVSize()
print("Number of unknowns:", Size)

ess_bdr = mfem.intArray(mesh.bdr_attributes.Max())
ess_bdr.Assign(0) 
ess_bdr[0] = 1 # Left boundary,  Dirichlet condition.
ess_bdr[1] = 1 # Right boundary

mu_vals, w_vals = gauss_legendre_dirs(N_ang)

#E_arr, E_grid_arr, xs_t_arr, xs_s_arr, S_arr = read_data(50)
S_arr = np.linspace(0, 2, nE + 1)
E_arr = np.linspace(1, 0.01, nE+1)
xs_t_coeff = TotalXSCoefficient(5)
xs_s_coeff = ScatteringXSCoefficient(5)
S_coeff    = StoppingPowerCoefficient(1)
dS_dE_arr = compute_S_derivative(E_arr, S_arr) 
dS_dE_coeff = StoppingPowerDerivativeCoefficient(dS_dE_arr, E_start, E_end)
q_coeff = ConstantCoefficient(100)
inflow_coeff = ConstantCoefficient(1.0)
assign_boundary_attributes(mesh, x_start, E_start, x_end, tol=1e-6)

Number of unknowns: 1200


In [7]:
# psi = mfem.GridFunction(fes)
#psi.Assign(1.0)
#psi.ProjectBdrCoefficient(inflow_coeff, ess_bdr)
psi_mu_list = []

for mu, w in zip(mu_vals, w_vals):
    print("  Solving for mu =", mu)
    mu_coeff = constant(mu)
    marker = get_marker_for_mu(mesh, mu)
    v_coeff = VelocityCoefficient(mu, S_arr, E_start, E_end)

    a = mfem.BilinearForm(fes)
    a.AddDomainIntegrator(mfem.ConvectionIntegrator(v_coeff))
    a.AddDomainIntegrator(mfem.MassIntegrator(xs_t_coeff))
    a.AddDomainIntegrator(mfem.MassIntegrator(dS_dE_coeff))
    a.AddInteriorFaceIntegrator(mfem.TransposeIntegrator(mfem.DGTraceIntegrator(v_coeff, 1.0, -0.5)))
    a.AddBdrFaceIntegrator(mfem.TransposeIntegrator(mfem.DGTraceIntegrator(v_coeff, 1.0, -0.5)))
    a.Assemble()
    a.Finalize()
    A = a.SpMat()

    # Set up the linear form b for the RHS
    b = mfem.LinearForm(fes)
    b.AddDomainIntegrator(mfem.DomainLFIntegrator(q_coeff))
    b.AddBdrFaceIntegrator(mfem.BoundaryFlowIntegrator(inflow_coeff, v_coeff, -1.0), marker)
    #b.AddDomainIntegrator(mfem.DomainLFIntegrator(scattering_source))
    b.Assemble()

    psi = mfem.GridFunction(fes)
    psi.Assign(1.0) 
    prec = mfem.GSSmoother(A)
    solver = mfem.GMRESSolver()
    solver.SetOperator(A)
    solver.SetPreconditioner(prec)
    solver.SetRelTol(1e-12)
    solver.SetAbsTol(1e-12)
    solver.SetMaxIter(500)
    solver.SetKDim(30)
    solver.SetPrintLevel(0)
    solver.Mult(b, psi)

    print("    GMRES (mu =", mu, "): iterations =", solver.GetNumIterations(),
          "final norm =", solver.GetFinalNorm())
    
    # Append the angular solution for mu
    psi_mu_list.append((mu, w, psi))
    
    # Save each angular solution and corresponding mesh.
    psi.Save("psi_mu_list{:.3f}.gf".format(mu))

phi_new = mfem.GridFunction(fes)
phi_new.Assign(0.0)

for mu, w, psi in psi_mu_list:
    phi_new.Add(w, psi)

phi_new.Save("phi_newww.gf")



  Solving for mu = -0.8611363115940526
    GMRES (mu = -0.8611363115940526 ): iterations = 19 final norm = 485880.88467697875
  Solving for mu = -0.33998104358485626
    GMRES (mu = -0.33998104358485626 ): iterations = 6 final norm = 4723839184490273.0
  Solving for mu = 0.33998104358485626
    GMRES (mu = 0.33998104358485626 ): iterations = 500 final norm = 843605587781.5165
  Solving for mu = 0.8611363115940526
GMRES: Number of iterations:     GMRES (mu = 0.8611363115940526 ): iterations = 500 final norm = 217.30394221998725
500
GMRES: No convergence!
GMRES: Number of iterations: 500
GMRES: No convergence!
